In [1]:
!pip install boto3

You should consider upgrading via the 'C:\Users\Bryan\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install load_dotenv

You should consider upgrading via the 'C:\Users\Bryan\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import boto3
import warnings
import os
from dotenv import load_dotenv
warnings.filterwarnings("ignore")

In [4]:
load_dotenv('env.s3creds')

True

s3 = boto3.resource(
	service_name='s3',
	aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)

In [5]:
s3 = boto3.resource(
	service_name='s3',
	aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)

In [6]:
# Verify that the variables are correctly retrieved (optional)
print(f"Client Access: {s3}")


Client Access: s3.ServiceResource()


In [7]:
# List all buckets
for bucket in s3.buckets.all():
    print(bucket.name)

prj4


In [8]:
for obj in s3.Bucket('prj4').objects.all():
    print(obj)

s3.ObjectSummary(bucket_name='prj4', key='StormEvents_2014.csv')
s3.ObjectSummary(bucket_name='prj4', key='eaglei_outages_2014.csv')
s3.ObjectSummary(bucket_name='prj4', key='stormOutages_df.csv')


In [9]:
obj = s3.Bucket('prj4').Object('stormOutages_df.csv').get()
file = pd.read_csv(obj['Body'], index_col=0)

df = pd.DataFrame(file)

In [10]:
df.head(5)


,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out
END_YEARMONTH,,,,,,,
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52


In [11]:
df = df.assign(fips_code=df.index)

In [12]:
df.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code
END_YEARMONTH,,,,,,,,
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411


In [13]:
df = df.reset_index(drop=True)

In [14]:
df.shape

(1007790, 8)

In [15]:
customer_counts = df['customers_out'].value_counts()

# Print the result
print(customer_counts)

1        261368
2         91848
3         48512
4         35958
5         32465
          ...  
10674         1
4032          1
4010          1
4004          1
4196          1
Name: customers_out, Length: 7944, dtype: int64


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007790 entries, 0 to 1007789
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   EVENT_ID       1007790 non-null  int64 
 1   STATE          1007790 non-null  object
 2   county         1007790 non-null  object
 3   YEAR           1007790 non-null  int64 
 4   EVENT_TYPE     1007790 non-null  object
 5   FIPS           1007790 non-null  int64 
 6   customers_out  1007790 non-null  int64 
 7   fips_code      1007790 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 61.5+ MB


In [17]:
df.columns

Index(['EVENT_ID', 'STATE', 'county', 'YEAR', 'EVENT_TYPE', 'FIPS',
       'customers_out', 'fips_code'],
      dtype='object')

In [18]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

### Step 3: Split the data into training and testing datasets by using `train_test_split`.

Due to the dataset containing many rows of value "1" for the amount of customers out, that value cannot be used to train in the data set. We will clean the data to remove the counts of "1" for customers out in the data. 

Create the labels set (`y`)  from the customers out value column, and then create the features (`X`) DataFrame from the remaining columns.

In [72]:
df_filtered = df.query("customers_out > 1")
df_filtered.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code
0,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
1,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
2,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
3,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
4,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411


In [73]:
df_filtered['fips_code'] = df_filtered['fips_code'].astype(str)
df_filtered['month'] = df_filtered['fips_code'].str[-2:]
df_filtered.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code,month
0,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11
1,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11
2,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11
3,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11
4,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11


### Step 2: Create the labels set (`y`)  from the “customers_out” column, and then create the features (`X`) DataFrame from the remaining columns.

### we need to convert string/char data types to a coded numerical value in order to translate the data into a usable format for logistical regression. 
## we will take all pertinent data such in str format such as "STATE" and "county" then concatenate them so they are unique.  -- NVM We can use FIPS !! 
# ## once they are merged, we can assign it a numerical value in alphabetical order to render in the model as a "Coded" location based value. --AGAIN  -- NVM We can use FIPS !! 
## EVENT_ID is garbage probably, DELETE 
### EVENT_TYPE Should be classified by intensity of the storm type in ASC order. yay :) 

In [74]:
event_type_counts = df['EVENT_TYPE'].value_counts()
event_type_counts

Winter Weather             205698
Thunderstorm Wind          133259
Winter Storm               106094
Flood                       88426
Strong Wind                 76272
Heavy Snow                  60019
Heavy Rain                  52016
Flash Flood                 50925
Dense Fog                   41227
High Wind                   38363
Drought                     33961
Hail                        26071
Lake-Effect Snow            24936
Tornado                     17513
Cold/Wind Chill              9538
Frost/Freeze                 8476
Funnel Cloud                 8263
Lightning                    7851
Coastal Flood                7829
Debris Flow                  3393
Seiche                       2997
Ice Storm                    2806
Wildfire                     1518
Extreme Cold/Wind Chill       279
Avalanche                      50
Blizzard                        8
Sleet                           2
Name: EVENT_TYPE, dtype: int64

In [75]:
# Define a dictionary to map weather events to scores
event_scores = {
    "Winter Weather": 2,  # Moderate impact
    "Thunderstorm Wind": 5,  # Can cause damage
    "Winter Storm": 7,  # Significant disruptions
    "Flood": 8,  # Major infrastructure damage
    "Strong Wind": 4,  # Potential for damage
    "Heavy Snow": 6,  # Disruptions to travel and infrastructure
    "Heavy Rain": 5,  # Can cause flooding and disruptions
    "Flash Flood": 8,  # Life-threatening and destructive
    "Dense Fog": 3,  # Can cause travel delays and accidents
    "High Wind": 4,  # Potential for damage
    "Drought": 7,  # Long-term impact on agriculture and water resources
    "Hail": 5,  # Can damage property and crops
    "Lake-Effect Snow": 6,  # Can cause heavy snowfall in localized areas
    "Tornado": 9,  # Devastating impact on property and lives
    "Cold/Wind Chill": 3,  # Discomfort and health risks for vulnerable populations
    "Frost/Freeze": 2,  # Can damage crops and vegetation
    "Funnel Cloud": 1,  # Not always associated with a tornado
    "Lightning": 3,  # Can cause fires and injuries
    "Coastal Flood": 8,  # Damage to coastal infrastructure and property
    "Debris Flow": 8,  # Rapid movement of debris can be destructive
    "Seiche": 2,  # Water level fluctuations, typically less impactful
    "Ice Storm": 7,  # Power outages and damage to infrastructure
    "Wildfire": 9,  # Widespread destruction of property and ecological damage
    "Extreme Cold/Wind Chill": 8,  # Life-threatening conditions
    "Avalanche": 9,  # Destructive and can cause fatalities
    "Blizzard": 8,  # Severe travel disruptions and potential infrastructure damage
    "Sleet": 3  # Mix of rain and snow, can cause slippery conditions
}

# Assign scores to the "EVENT_TYPE" column and create a new "domain" column
df_filtered["storm_intensity"] = df_filtered["EVENT_TYPE"].apply(lambda event: event_scores.get(event, 1))  # Assign 1 for missing values

# Print the modified DataFrame (optional)
df_filtered.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code,month,storm_intensity
0,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11,5
1,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11,5
2,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5
3,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5
4,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5


In [77]:

# Adding some identifier values for the model to interpret "outage events" with a general threshold of 500 customers or more to indicate an outage event.
df_filtered['outage_event'] = df_filtered['customers_out'].apply(lambda x: 0 if x <= 500 else 1)

df_filtered.head(20)


,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code,month,storm_intensity,outage_event
0,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11,5,0
1,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11,5,0
2,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5,0
3,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5,0
4,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5,0
5,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,2918,201411,11,5,1
6,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,2928,201411,11,5,1
7,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,2940,201411,11,5,1
8,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,2944,201411,11,5,1
9,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,2950,201411,11,5,1


In [78]:
sample_df = df_filtered.sample(frac=0.1, random_state=42)  # Adjust the fraction as needed

In [80]:
# Separate the data into labels and features

# Separate the y variable, the labels
y=sample_df['customers_out']

# Separate the X variables from the features
x = sample_df.drop(columns=['customers_out', 'EVENT_ID', 'fips_code', 'STATE', 'county', 'EVENT_TYPE'])

In [81]:
y.head()

653158        2
4598         10
552883    13717
374712      268
784124      576
Name: customers_out, dtype: int64

In [82]:
y.value_counts()

2        9128
3        4789
4        3657
5        3219
6        2169
         ... 
8202        1
24458       1
11642       1
2832        1
1686        1
Name: customers_out, Length: 3496, dtype: int64

In [83]:
x.head()

,YEAR,FIPS,month,storm_intensity,outage_event
653158,2014,48439201412,12,5,0
4598,2014,45063201411,11,5,0
552883,2014,23027201411,11,7,1
374712,2014,6065201412,12,1,0
784124,2014,48309201411,11,5,1


### Step 3: Split the data into training and testing datasets by using `train_test_split`.

In [29]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split
### We had to remove the stratify = y argument due to multiple values in our testing data of having only one occurrence. The testing/training of this method
### requires multiple occurrences of the customer out value to train against, and one example will not allow the tool to analyze and understand the difference and 
## potential values to expect given the other values/parameters of our "X" data 
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1)
X_train.shape

(33828, 5)

In [30]:
y_train.shape

(33828,)

In [31]:
y_test.shape

(11277,)

## Create a Logistic Regression Model with the Original Data
#####  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [85]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
Logistic_regr = LogisticRegression(solver='lbfgs',max_iter=10,random_state=1)
Logistic_regr

# Fit the model using training data
Logistic_regr.fit(X_train,y_train)

LogisticRegression(max_iter=10, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [86]:
# Make a prediction using the testing data
prediction = Logistic_regr.predict(X_test)
prediction_df = pd.DataFrame({"Prediction": prediction, "Actual": y_test}).reset_index(drop=True)
prediction_df.head()

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [88]:
prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11277 entries, 0 to 11276
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Prediction  11277 non-null  int64
 1   Actual      11277 non-null  int64
dtypes: int64(2)
memory usage: 176.3 KB


### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.

* Print the classification report.

In [89]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd

In [90]:
# Generate a confusion matrix for the model
conf_mat = confusion_matrix(y_test, prediction)
#conf_mat_df = pd.DataFrame(conf_mat, index= [["Base 0 ", "Base 1 "], columns=["Predicted 0", "Predicted 1"]])
accuracy_score = accuracy_score(y_test, prediction)

In [91]:
from sklearn.metrics import classification_report

# Assuming model is your trained model and X_test is your test dataset
y_true = y_test  # True labels
y_pred = Logistic_regr.predict(X_test)  # Predicted labels

# Ensure y_pred is an array
y_pred = y_pred.tolist() if not isinstance(y_pred, (list, np.ndarray)) else y_pred

# Print the classification report for the model
# Displaying results
print("Confusion Matrix Chart")
display(conf_mat)
print(f"Accuracy Score : {accuracy_score}")
print("Classifications")
report = classification_report(y_true, y_pred)
print(report)

Confusion Matrix Chart


array([[9516,    0],
       [1761,    0]], dtype=int64)

Accuracy Score : 0.8438414471934025
Classifications
              precision    recall  f1-score   support

           0       0.84      1.00      0.92      9516
           1       0.00      0.00      0.00      1761

    accuracy                           0.84     11277
   macro avg       0.42      0.50      0.46     11277
weighted avg       0.71      0.84      0.77     11277



**Question:** How well does the logistic regression model predict the occurrence and severity of a storm, by customers out across the US?

**Answer:** 

Not bad..

The model is attempting to predict the likeliness of # of power outage "events" by loation and date, however, the dataset is limited in the variables being measured for this model's decision. If this dataset were narrowed down to a single location/state, with more years/data points such as windspeed, tide levels,and amount of precip for more weather points, as well as informatoin such as utility construction type (OH vs UG), it  would help in determining potential outage events and severity. In conclusion, this is only a small portion of the data required for a model to predict # of customers out for a storm event, but can be expanded on for better modeling and accuracy.

Another limitatoin with this model type (logistic regression) is that it's best used to determine the likeliness of something to occur, true/false. We simplified our data points by labeling them "outage event" vs "non-outage event" to test it's feasibility. We believe this analysis would be MUCH more helpful if we had the extra data points mentioned above, and altered our model to run multi-linear regression. If we continued to use our Y variable for the amount of customers out, then used the X variables of storm intensity, month, outage event,  and the extra data points mentioned previously, our model may become more accurate and useful to determine serverity and likliness.

Let's try to run this data through a simple linear regression model to see how it performs on predicting customers out....

In [94]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define the target variable (y) and the feature variables (X)
y = sample_df['customers_out']
X = sample_df.drop(columns=['customers_out', 'EVENT_ID', 'fips_code', 'STATE', 'county', 'EVENT_TYPE'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Instantiate the Linear Regression model
linear_reg = LinearRegression()

# Fit the model using the training data
linear_reg.fit(X_train, y_train)

# Make predictions using the testing data
y_pred = linear_reg.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Display coefficients and intercept
print("Coefficients:", linear_reg.coef_)
print("Intercept:", linear_reg.intercept_)

# Create a DataFrame to compare actual vs predicted values
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
comparison_df.head()


Mean Absolute Error: 448.25264116692125
Mean Squared Error: 5307801.746031697
R-squared: 0.16861471495343416
Coefficients: [ 0.00000000e+00  1.39783551e-09 -2.43892897e+02  2.44436224e+01
  3.21452922e+03]
Intercept: 2687.625800479301


,Actual,Predicted
883866,42,-148.904107
693405,310,121.088103
521598,2,-132.202769
308213,2,-100.053206
802807,2,-0.687326
